# Lab 2. Customer Insights Agent for Financial Visualization Explanation

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent helps bank operators understand and explain financial data visualizations to their customers. It analyzes the underlying data of existing visualizations and provides clear, insightful explanations that bank operators can share with customers.

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if a visualization is too complex to explain, bank operators can ask the agent to create a support ticket, to get a human financial advisor to help with more detailed explanations.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/customer_insights_agent.png)

## Setup

Make sure that your boto3 version is the latest

If not, return to [notebook 1](../1-data-analytics/1_data_analytics_agent.ipynb) and run the Setup block again.

In [ ]:
!pip freeze | grep boto3

Get your workshop ID / resource suffix. If not found, return to [notebook 1](../1-data-analytics/1_data_analytics_agent.ipynb) and run the Setup block again.

In [ ]:
import os

def get_workshop_id():
    workshop_id_file = '../.workshop_id'
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        return None
    
workshop_id = get_workshop_id()
resource_suffix = f"{workshop_id}"

if workshop_id is None:
    print("No workshop ID found. Please run the Setup script in notebook 1.")
else:
    print("Your resource suffix is", resource_suffix)

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create our second agent.

In [ ]:
import boto3
import os

from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import time

account_id = boto3.client('sts').get_caller_identity().get('Account')

agent_foundation_model = [
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [ ]:
curr_month = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)

insights_agent_name = f"insights-{resource_suffix}"

insights_lambda_name = f"fn-insights-agent-{resource_suffix}"

insights_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{insights_agent_name}'

dynamodb_table = f"{insights_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{insights_agent_name}-kb'

knowledge_base_description = "KB containing information on financial visualization explanation"
bucket_name = f'insights-kb-{account_id}-{resource_suffix}'

### Importing helper functions

Let's import the helper functions that we'll use to create our agent. These functions are defined in the `utils` directory and include:

- `create_agent`: Create a new agent on Amazon Bedrock.
- `create_knowledge_base`: Create a new knowledge base on Amazon Bedrock.
- `create_s3_bucket`: Create a new S3 bucket to store the knowledge base documents.
- `upload_to_s3`: Upload files to S3.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Creating Knowledge Base

Next we will create a knowledge base with information about financial visualization explanation. This knowledge base will help our agent provide better explanations for different types of financial visualizations.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of guidelines for explaining financial visualizations to customers.

**This creation process takes several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

## Create Knowledge Base Documents

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [ ]:
path = "kb_documents"

# Check whether the specified path exists or not
is_exist = os.path.exists(path)
if not is_exist:
   # Create a new directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [ ]:
def invoke_bedrock_generate_finance_files(prompt):
    bedrock_runtime = boto3.client(service_name='bedrock-runtime')
    
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    })
    
    modelId = 'anthropic.claude-3-sonnet-20240229-v1'
    accept = 'application/json'
    contentType = 'application/json'
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    
    response_body = json.loads(response.get('body').read())
    
    return response_body['content'][0]['text']

def write_file(file_name, content):
    f = open(file_name, "w")
    f.write(content)
    f.close

Generating a file with information on financial visualization explanation.

In [ ]:
# We already have the knowledge base document prepared
print("Using existing knowledge base document: kb_documents/customer-insights-info.txt")

In [ ]:
%%time
kb.upload_documents_to_s3(
    bucket_name,
    "kb_documents/customer-insights-info.txt"
)

In [ ]:
%%time
kb.synchronize_data(ds_id)

## Creating the Customer Insights Agent

Now we'll create the Customer Insights Agent that will help bank operators explain financial visualizations to their customers.

In [ ]:
%%time
insights_agent = agents.create_agent(
    insights_agent_name,
    "Customer Insights Agent for Financial Visualization Explanation",
    agent_foundation_model[0],
    "agent_prompt.txt",
    insights_agent_role_name
)

### Associate Knowledge Base with Agent

Now we'll associate the knowledge base with the agent so it can access information about financial visualization explanation.

In [ ]:
%%time
agents.associate_kb_with_agent(
    insights_agent[0],
    kb_id,
    "Financial visualization explanation knowledge base",
    "Contains information about how to explain different types of financial visualizations"
)

### Create Lambda Function for Agent Actions

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group.

In [ ]:
%%time
%%writefile customer_insights.py
# This file is already created separately

In [ ]:
%%time
insights_lambda_arn = agents.create_lambda_function(
    insights_lambda_name,
    "customer_insights.py",
    dynamoDB_args
)

### Define Agent Actions

Next we will define the available actions that an agent can perform using Function Details. These actions will allow the agent to explain different types of financial visualizations and recommend financial products.

In [ ]:
%%time
agents.add_action_group_with_lambda(
    insights_agent[0],
    "agent_api_definition.json",
    insights_lambda_arn
)

## Loading Sample Visualization Data

Now that we've created our agent, let's load some sample visualization data that the agent can explain.

In [ ]:
# We already have the sample data prepared
print("Using existing sample visualization data: 2_user_sample_data.json")

with open("2_user_sample_data.json") as f:
    visualizations = json.load(f)
    
print(f"Loaded {len(visualizations)} sample visualizations")

## Visualizing and Testing the Customer Insights Agent

Now that we've created our agent, let's visualize the sample data and test how the agent explains each visualization.

## Visualizing the Sample Data

Before testing our agent, let's visualize the sample data to better understand what the agent will be explaining.

In [ ]:
# Install matplotlib if not already installed
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

# Function to format y-axis as currency
def currency_formatter(x, pos):
    return f'${int(x):,}'

# Set style for all plots
plt.style.use('ggplot')
formatter = FuncFormatter(currency_formatter)

### 1. Spending Trend Visualization

This visualization shows the customer's spending trend over a 6-month period.

In [ ]:
# Extract spending trend data
spending_trend = visualizations[0]
months = [point['month'].split('/')[1] for point in spending_trend['data_points']]
amounts = [point['amount'] for point in spending_trend['data_points']]

# Create the spending trend line chart
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(months, amounts, marker='o', linewidth=2, markersize=8)
ax.set_title(spending_trend['metadata']['chart_title'], fontsize=16)
ax.set_xlabel(spending_trend['metadata']['x_axis_label'], fontsize=12)
ax.set_ylabel(spending_trend['metadata']['y_axis_label'], fontsize=12)
ax.yaxis.set_major_formatter(formatter)
ax.grid(True, linestyle='--', alpha=0.7)

# Add annotations for key points
max_point = max(amounts)
max_idx = amounts.index(max_point)
ax.annotate(f'${max_point}', 
            xy=(months[max_idx], max_point), 
            xytext=(0, 15),
            textcoords='offset points',
            ha='center',
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'))

plt.tight_layout()
plt.savefig('img/spending_trend.png')
plt.show()

In [ ]:
# Extract category data
categories = [cat['name'] for cat in spending_trend['categories']]
percentages = [cat['percentage'] for cat in spending_trend['categories']]

# Create the spending by category pie chart
fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts, autotexts = ax.pie(percentages, 
                                  autopct='%1.1f%%',
                                  textprops=dict(color="w"),
                                  startangle=90)

# Draw a circle at the center to make it a donut chart
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')  
ax.set_title('Spending by Category', fontsize=16)

# Add legend
ax.legend(wedges, categories, title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig('img/spending_categories.png')
plt.show()

Now let's see how our Customer Insights Agent explains this spending trend visualization:

In [ ]:
%%time
spending_trend_data = json.dumps(visualizations[0])
response = agents.invoke(
    f"Explain this spending trend visualization for customer with ID 1: {spending_trend_data}", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

### 2. Investment Allocation Visualization

This visualization shows how the customer's investments are allocated across different asset classes.

In [ ]:
# Extract investment allocation data
investment = visualizations[1]
asset_classes = [asset['asset_class'] for asset in investment['data_points']]
percentages = [asset['percentage'] for asset in investment['data_points']]

# Create the investment allocation pie chart
fig, ax = plt.subplots(figsize=(10, 8))

# Use custom colors for different asset classes
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
wedges, texts, autotexts = ax.pie(percentages, 
                                  colors=colors,
                                  autopct='%1.1f%%',
                                  textprops=dict(color="k"),
                                  startangle=90)

# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')  
ax.set_title(investment['metadata']['chart_title'], fontsize=16)

# Add legend
ax.legend(wedges, asset_classes, title="Asset Classes", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig('img/investment_allocation.png')
plt.show()

Now let's see how our Customer Insights Agent explains this investment allocation visualization:

In [ ]:
%%time
investment_allocation_data = json.dumps(visualizations[1])
response = agents.invoke(
    f"What does this investment allocation chart show for customer with ID 1? {investment_allocation_data}", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

### 3. Cash Flow Visualization

This visualization compares income and expenses over a 3-month period.

In [ ]:
# Extract cash flow data
cash_flow = visualizations[2]
months = [point['month'].split('/')[1] for point in cash_flow['data_points']]
income = [point['income'] for point in cash_flow['data_points']]
expenses = [point['expenses'] for point in cash_flow['data_points']]

# Set width of bars
barWidth = 0.35
r1 = np.arange(len(months))
r2 = [x + barWidth for x in r1]

# Create the cash flow bar chart
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(r1, income, width=barWidth, label='Income', color='#5cb85c')
ax.bar(r2, expenses, width=barWidth, label='Expenses', color='#d9534f')

# Add labels and title
ax.set_title(cash_flow['metadata']['chart_title'], fontsize=16)
ax.set_xlabel(cash_flow['metadata']['x_axis_label'], fontsize=12)
ax.set_ylabel(cash_flow['metadata']['y_axis_label'], fontsize=12)
ax.set_xticks([r + barWidth/2 for r in range(len(months))])
ax.set_xticklabels(months)
ax.yaxis.set_major_formatter(formatter)

# Add net cash flow values above the bars
for i in range(len(months)):
    net = income[i] - expenses[i]
    color = 'green' if net >= 0 else 'red'
    ax.annotate(f'Net: ${net}', 
                xy=((r1[i] + r2[i])/2, max(income[i], expenses[i]) + 200),
                ha='center',
                va='bottom',
                color=color,
                fontweight='bold')

# Add legend
ax.legend()

plt.tight_layout()
plt.savefig('img/cash_flow.png')
plt.show()

Now let's see how our Customer Insights Agent explains this cash flow visualization:

In [ ]:
%%time
cash_flow_data = json.dumps(visualizations[2])
response = agents.invoke(
    f"Explain this cash flow visualization for customer with ID 2: {cash_flow_data}", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

### 4. Budget Performance Visualization

This visualization compares planned versus actual spending across different categories.

In [ ]:
# Extract budget performance data
budget = visualizations[3]
categories = [item['category'] for item in budget['data_points']]
planned = [item['planned'] for item in budget['data_points']]
actual = [item['actual'] for item in budget['data_points']]

# Set width of bars
barWidth = 0.35
r1 = np.arange(len(categories))
r2 = [x + barWidth for x in r1]

# Create the budget performance bar chart
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(r1, planned, width=barWidth, label='Planned', color='#5bc0de')
ax.bar(r2, actual, width=barWidth, label='Actual', color='#f0ad4e')

# Add labels and title
ax.set_title(budget['metadata']['chart_title'], fontsize=16)
ax.set_xlabel(budget['metadata']['x_axis_label'], fontsize=12)
ax.set_ylabel(budget['metadata']['y_axis_label'], fontsize=12)
ax.set_xticks([r + barWidth/2 for r in range(len(categories))])
ax.set_xticklabels(categories)
ax.yaxis.set_major_formatter(formatter)

# Add variance percentages above the bars
for i in range(len(categories)):
    variance = ((actual[i] - planned[i]) / planned[i]) * 100
    color = 'red' if variance > 0 else 'green'
    ax.annotate(f'{variance:.1f}%', 
                xy=((r1[i] + r2[i])/2, max(planned[i], actual[i]) + 50),
                ha='center',
                va='bottom',
                color=color,
                fontweight='bold')

# Add legend
ax.legend()

plt.tight_layout()
plt.savefig('img/budget_performance.png')
plt.show()

Now let's see how our Customer Insights Agent explains this budget performance visualization:

In [ ]:
%%time
budget_performance_data = json.dumps(visualizations[3])
response = agents.invoke(
    f"What does this budget performance visualization tell us about customer with ID 3? {budget_performance_data}", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

## Testing Additional Agent Capabilities

Beyond explaining visualizations, our Customer Insights Agent can also recommend financial products and manage support tickets.

### Testing Support Ticket Creation

Let's test the agent's ability to create a support ticket for visualization explanation assistance.

In [ ]:
%%time
response = agents.invoke(
    "I need help understanding a complex visualization for customer with ID 1. Can you create a support ticket?", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

### Testing Support Ticket Retrieval

Let's test the agent's ability to retrieve support tickets for a customer.

In [ ]:
%%time
response = agents.invoke(
    "Can I get all tickets that I have? My customer id is 1", 
    insights_agent[0], enable_trace=True
)
print("====================")
print(response)

## Create Agent Alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
insights_agent_alias_id, insights_agent_alias_arn = agents.create_agent_alias(
    insights_agent[0], 'v1'
)
insights_agent_id = insights_agent[0]

Store environment variables to be used on next notebooks.

In [ ]:
insights_agent_arn = agents.get_agent_arn_by_name(insights_agent_name)
insights_kb = knowledge_base_name
insights_dynamodb = dynamodb_table

%store insights_agent_arn
%store insights_agent_alias_arn
%store insights_agent_alias_id
%store insights_lambda_name
%store insights_agent_name
%store insights_agent_id
%store insights_kb
%store insights_dynamodb

In [ ]:
insights_agent_arn, insights_agent_alias_arn, insights_agent_alias_id

## Next Steps
Congratulations! We've now created a Customer Insights Agent that can explain financial visualizations. Next we will create our Risk & Compliance Agent.